# Implement Grouped Query Attention from Scratch

### Problem Statement

Standard Multi-Head Attention (MHA) assigns a separate key and value projection to each attention head. **Grouped Query Attention (GQA)** is a more efficient variant where multiple query heads share the same key-value heads.

### Background: Why GQA?

During autoregressive generation, the KV cache can become a memory bottleneck:
- For a 70B model with 64 heads and 8K context, MHA needs ~20GB just for KV cache
- GQA reduces this by sharing K/V across multiple query heads

### The Key Insight

| Attention Type | Query Heads | KV Heads | KV Cache Size |
|---------------|-------------|----------|---------------|
| MHA | 32 | 32 | 32 * seq * d_head * 2 |
| GQA (8 groups) | 32 | 8 | **8 * seq * d_head * 2** |
| MQA (1 group) | 32 | 1 | **1 * seq * d_head * 2** |

**Real-world examples:**
- LLaMA-2 70B: 64 query heads, 8 KV heads = **8x memory reduction**
- Mistral 7B: 32 query heads, 8 KV heads = **4x memory reduction**

### Learning Path

1. **Part 1**: Mask creation (causal, padding, KV cache)
2. **Part 2**: Core GQA mechanism - understand the repeat_interleave for grouped K/V
3. **Part 3**: GQA Self-Attention - Q, K, V from projections of single input x
4. **Part 4**: GQA with KV Cache - see the actual memory savings

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Part 1: Attention Mask Creation

The mask functions are the same as standard attention. For GQA, the mask broadcasts across all heads.

Mask convention: **True = masked (cannot attend), False = can attend**

In [ ]:
def create_causal_mask(seq_len_q: int, seq_len_k: int = None, device=None) -> torch.Tensor:
    """
    Create a causal (lower-triangular) attention mask for autoregressive models.
    
    Args:
        seq_len_q: Query sequence length
        seq_len_k: Key sequence length (defaults to seq_len_q)
        device: Device to create tensor on
    
    Returns:
        mask: Boolean tensor of shape (seq_len_q, seq_len_k)
              True = position should be MASKED (cannot attend)
              False = position can be attended to
    
    Example for seq_len=4:
        Q\\K   0      1      2      3
        0   [False, True,  True,  True ]   # Query 0 attends only to Key 0
        1   [False, False, True,  True ]   # Query 1 attends to Keys 0-1
        2   [False, False, False, True ]   # Query 2 attends to Keys 0-2
        3   [False, False, False, False]   # Query 3 attends to Keys 0-3
    """
    pass

In [ ]:
def create_padding_mask(lengths: torch.Tensor, max_len: int) -> torch.Tensor:
    """
    Create a padding mask for variable-length sequences.
    
    Args:
        lengths: Tensor of shape (batch_size,) containing actual sequence lengths
        max_len: Maximum sequence length (padded length)
    
    Returns:
        mask: Boolean tensor of shape (batch_size, max_len)
              True = padding position (should be MASKED)
              False = real token (can be attended to)
    
    Example:
        lengths = [3, 5, 2], max_len = 5
        
        Returns:
        [[False, False, False, True,  True ],   # seq 0: tokens 0-2 real, 3-4 padding
         [False, False, False, False, False],   # seq 1: all 5 tokens real
         [False, False, True,  True,  True ]]   # seq 2: tokens 0-1 real, 2-4 padding
    """
    pass

In [ ]:
def create_attention_mask(
    seq_len_q: int,
    seq_len_k: int = None,
    is_causal: bool = True,
    key_padding_lengths: torch.Tensor = None,
    device=None
) -> torch.Tensor:
    """
    Create a combined attention mask for GQA.
    
    Args:
        seq_len_q: Query sequence length
        seq_len_k: Key sequence length
        is_causal: Whether to apply causal masking
        key_padding_lengths: If provided, actual lengths of key sequences (batch_size,)
        device: Device to create tensor on
    
    Returns:
        mask with shape suitable for broadcasting to (batch, num_heads, seq_q, seq_k)
    """
    pass

In [ ]:
def create_causal_mask_with_cache(
    seq_len_q: int,
    seq_len_k: int,
    cache_len: int,
    device=None
) -> torch.Tensor:
    """
    Create a causal mask for attention with KV cache.
    New queries can attend to all cached positions.
    
    Args:
        seq_len_q: Number of new query tokens (typically 1 during generation)
        seq_len_k: Total key length (cache_len + seq_len_q)
        cache_len: Number of cached tokens
        device: Device to create tensor on
    
    Returns:
        mask: Boolean tensor of shape (seq_len_q, seq_len_k)
    
    Example: cache_len=3, seq_len_q=2, seq_len_k=5
        Keys:    [cached0, cached1, cached2, new0, new1]
        
        Q\\K      c0     c1     c2    new0   new1
        new0  [False, False, False, False, True ]  # new0 attends to cache + itself
        new1  [False, False, False, False, False]  # new1 attends to cache + new0 + itself
    """
    pass

In [ ]:
# Test mask creation
mask = create_causal_mask(4)
print("Causal mask (4x4):")
print(mask)

# Test with padding
lengths = torch.tensor([4, 3])
mask_combined = create_attention_mask(4, is_causal=True, key_padding_lengths=lengths)
print(f"\nCombined mask shape: {mask_combined.shape}")
print("\n\u2713 Mask tests passed!")

## Part 2: Core GQA Mechanism

The key difference from standard MHA:
- Q has `num_query_heads` heads
- K and V have `num_kv_heads` heads (fewer!)
- We expand K, V using `repeat_interleave` to match Q's head count

In [ ]:
torch.manual_seed(42)

batch_size = 2
seq_len = 8
d_model = 64
num_query_heads = 8
num_kv_heads = 2  # 4x fewer KV heads than query heads
d_head = d_model // num_query_heads

print(f"d_model={d_model}")
print(f"num_query_heads={num_query_heads}, num_kv_heads={num_kv_heads}")
print(f"d_head={d_head}")
print(f"Query heads per KV head: {num_query_heads // num_kv_heads}")
print(f"\nKV projection size: {num_kv_heads * d_head} (vs {d_model} for full MHA)")

In [ ]:
def grouped_query_attention_core(Q, K, V, num_query_heads, num_kv_heads, mask=None):
    """
    Core GQA computation (Q, K, V already projected and reshaped to heads).
    
    Args:
        Q: Query tensor (batch, num_query_heads, seq_len, d_head)
        K: Key tensor (batch, num_kv_heads, seq_len, d_head) - fewer heads!
        V: Value tensor (batch, num_kv_heads, seq_len, d_head) - fewer heads!
        num_query_heads: Number of query heads
        num_kv_heads: Number of key/value heads
        mask: Optional boolean attention mask (True = masked)
    
    Returns:
        output: GQA output (batch, num_query_heads, seq_len, d_head)
    
    Hints:
        - Use repeat_interleave to expand K, V to match query heads
        - The repeat factor is num_query_heads // num_kv_heads
        - After expansion, use standard scaled dot-product attention
    """
    pass

In [ ]:
# Test core GQA mechanism
Q = torch.randn(batch_size, num_query_heads, seq_len, d_head)
K = torch.randn(batch_size, num_kv_heads, seq_len, d_head)  # Fewer heads!
V = torch.randn(batch_size, num_kv_heads, seq_len, d_head)  # Fewer heads!

print(f"Q shape: {Q.shape} ({num_query_heads} heads)")
print(f"K shape: {K.shape} ({num_kv_heads} heads)")
print(f"V shape: {V.shape} ({num_kv_heads} heads)")

output = grouped_query_attention_core(Q, K, V, num_query_heads, num_kv_heads)
print(f"\nOutput shape: {output.shape}")

assert output.shape == Q.shape
print("\n\u2713 Core GQA mechanism test passed!")

In [ ]:
# Test with causal mask
causal_mask = create_causal_mask(seq_len)
output_causal = grouped_query_attention_core(Q, K, V, num_query_heads, num_kv_heads, mask=causal_mask)

print(f"Causal output shape: {output_causal.shape}")
print("\u2713 GQA with causal mask works!")

## Part 3: GQA Self-Attention

Full GQA where Q, K, V come from projections of a single input x.

Key difference from MHA:
- Q projection: `d_model -> d_model` (all query heads)
- K/V projection: `d_model -> kv_dim` where `kv_dim = num_kv_heads * d_head` (smaller!)

In [ ]:
class GroupedQuerySelfAttention(nn.Module):
    """
    Grouped Query Self-Attention where Q, K, V come from projections of the same input,
    but K and V have fewer heads than Q.
    
    Attributes:
        d_model: Total embedding dimension
        num_query_heads: Number of query heads
        num_kv_heads: Number of key/value heads (fewer than query heads)
        d_head: Dimension per head (d_model // num_query_heads)
        kv_dim: K/V projection dimension (num_kv_heads * d_head) - smaller than d_model!
        W_q: Query projection (d_model -> d_model)
        W_k, W_v: Key/Value projections (d_model -> kv_dim) - smaller!
        W_o: Output projection
    """
    
    def __init__(self, d_model: int, num_query_heads: int, num_kv_heads: int):
        super().__init__()
        pass
    
    def forward(
        self, 
        x: torch.Tensor, 
        is_causal: bool = False,
        key_padding_lengths: torch.Tensor = None
    ) -> torch.Tensor:
        """
        Args:
            x: Input tensor (batch, seq_len, d_model) - the residual stream
            is_causal: Whether to apply causal masking
            key_padding_lengths: If provided, actual lengths for padding mask
        
        Returns:
            output: GQA output (batch, seq_len, d_model)
        """
        pass

In [ ]:
# Test GQA Self-Attention
torch.manual_seed(42)

# Single input - the residual stream
x = torch.randn(batch_size, seq_len, d_model)
print(f"Input x shape: {x.shape}")
print("This single input will be projected to create Q, K, V")
print(f"Q: {d_model} dims ({num_query_heads} heads)")
print(f"K/V: {num_kv_heads * d_head} dims ({num_kv_heads} heads) - smaller!")

# Create GQA layer
gqa = GroupedQuerySelfAttention(d_model, num_query_heads, num_kv_heads)

# Forward pass (bidirectional)
output = gqa(x)
print(f"\nOutput shape: {output.shape}")

# Forward pass (causal)
output_causal = gqa(x, is_causal=True)
print(f"Causal output shape: {output_causal.shape}")

# Forward pass (with padding)
lengths = torch.tensor([8, 5])
output_padded = gqa(x, is_causal=True, key_padding_lengths=lengths)
print(f"Padded output shape: {output_padded.shape}")

# Count parameters
mha_params = d_model * d_model * 4  # Q, K, V, O each d_model x d_model
gqa_params = d_model * d_model + 2 * d_model * (num_kv_heads * d_head) + d_model * d_model
print(f"\nMHA projection params: {mha_params:,}")
print(f"GQA projection params: {gqa_params:,}")
print(f"Parameter reduction: {(1 - gqa_params/mha_params)*100:.1f}%")

assert output.shape == x.shape
print("\n\u2713 GQA Self-Attention test passed!")

## Part 4: GQA with KV Cache

### Why GQA Dramatically Reduces KV Cache Memory

The cache only needs to store K and V for `num_kv_heads`, not `num_query_heads`!

**Cache shape comparison:**
- MHA: `(batch, num_query_heads, seq_len, head_dim)`
- GQA: `(batch, num_kv_heads, seq_len, head_dim)` - smaller!

For LLaMA-2 70B with 8K context:
- MHA cache: ~20 GB
- GQA cache (8 KV heads): ~2.5 GB

In [ ]:
class GQAWithCache(nn.Module):
    """
    Grouped Query Self-Attention with KV Cache for efficient inference.
    
    Takes a single input x (the residual stream) and projects it to Q, K, V.
    The cache stores K and V with fewer heads than Q.
    
    Attributes:
        d_model: Total embedding dimension
        num_query_heads: Number of query heads
        num_kv_heads: Number of key/value heads (fewer than query heads)
        head_dim: Dimension per head (d_model // num_query_heads)
        kv_dim: K/V projection dimension (num_kv_heads * head_dim) - smaller than d_model!
        q_proj, k_proj, v_proj: Projections
        out_proj: Output projection
    """
    
    def __init__(self, d_model: int, num_query_heads: int, num_kv_heads: int):
        super().__init__()
        pass
    
    def forward(
        self,
        x: torch.Tensor,
        cache_k: torch.Tensor = None,
        cache_v: torch.Tensor = None,
        is_causal: bool = True,
    ) -> tuple:
        """
        Args:
            x: Input tensor (batch, seq_len, d_model) - the residual stream
            cache_k: Cached keys (batch, num_kv_heads, cached_len, head_dim)
            cache_v: Cached values (batch, num_kv_heads, cached_len, head_dim)
            is_causal: Whether to apply causal masking
        
        Returns:
            output: GQA output (batch, seq_len, d_model)
            new_cache_k: Updated key cache
            new_cache_v: Updated value cache
        """
        pass

In [ ]:
# Test GQA with KV Cache
print("=== Testing GQA with KV Cache ===")

torch.manual_seed(42)
batch_size = 2
d_model = 64
num_query_heads = 8
num_kv_heads = 2  # 4x fewer KV heads!

gqa_cached = GQAWithCache(d_model, num_query_heads, num_kv_heads)

# Step 1: Process prompt (3 tokens)
# x is the residual stream - Q, K, V all come from projections of x
prompt = torch.randn(batch_size, 3, d_model)
print(f"\nInput prompt shape: {prompt.shape}")
print(f"Q will have {num_query_heads} heads, K/V will have {num_kv_heads} heads")

out1, cache_k, cache_v = gqa_cached(prompt, None, None)
print(f"\nAfter prompt: cache shape = {cache_k.shape}")
print(f"  Note: only {num_kv_heads} KV heads cached, not {num_query_heads}!")

# Step 2: Generate token 4
new_token = torch.randn(batch_size, 1, d_model)
out2, cache_k, cache_v = gqa_cached(new_token, cache_k, cache_v)
print(f"After token 4: cache shape = {cache_k.shape}")

# Step 3: Generate token 5
new_token = torch.randn(batch_size, 1, d_model)
out3, cache_k, cache_v = gqa_cached(new_token, cache_k, cache_v)
print(f"After token 5: cache shape = {cache_k.shape}")

# Verify
head_dim = d_model // num_query_heads
assert cache_k.shape == (batch_size, num_kv_heads, 5, head_dim)
assert out3.shape == (batch_size, 1, d_model)

print("\n\u2713 GQA with KV Cache test passed!")

In [ ]:
# Memory comparison: GQA vs MHA
print("\n" + "="*60)
print("Memory Comparison: GQA vs MHA")
print("="*60)

mha_cache = batch_size * num_query_heads * 5 * head_dim * 2  # K + V
gqa_cache = batch_size * num_kv_heads * 5 * head_dim * 2     # K + V

print(f"MHA cache elements: {mha_cache:,} ({num_query_heads} heads)")
print(f"GQA cache elements: {gqa_cache:,} ({num_kv_heads} heads)")
print(f"Memory reduction: {mha_cache / gqa_cache:.1f}x")

print("\n" + "="*60)
print("Real-World Scaling (LLaMA-2 70B, 8K context)")
print("="*60)
print("MHA: 64 heads x 8K x 128 dim x 2 (K+V) x 80 layers x 2 bytes = ~20 GB")
print("GQA:  8 heads x 8K x 128 dim x 2 (K+V) x 80 layers x 2 bytes = ~2.5 GB")
print("Savings: 8x memory reduction!")

## Interview Tips

**Q: What's the difference between MHA, GQA, and MQA?**
A:
- MHA: Each query head has its own K and V heads (1:1 ratio)
- GQA: Multiple query heads share K/V heads (e.g., 8:1 ratio)
- MQA: All query heads share a single K/V head (all:1 ratio)

**Q: Why does GQA reduce memory but not compute?**
A: The K/V projections are smaller, saving some compute. But during attention, we expand K/V using repeat_interleave, so the actual attention computation is similar to MHA. The main savings are in KV cache memory.

**Q: When should you use GQA vs MHA?**
A: GQA is preferred for large models where KV cache memory is a bottleneck (inference with long contexts). For training or small models, MHA is fine.

**Q: How do you choose the number of KV heads?**
A: Common ratios are 4:1 to 8:1 (query:KV heads). LLaMA-2 70B uses 64:8, Mistral uses 32:8. The ratio depends on the tradeoff between quality and memory.

**Q: Does GQA hurt model quality?**
A: Slightly, but the tradeoff is usually worthwhile. GQA models achieve ~95-99% of MHA quality while using 4-8x less KV cache memory.